In [3]:
from sage.combinat.permutation import Permutation, Permutations
from sage.matrix.constructor import matrix, block_matrix
from sage.rings.integer_ring import ZZ
from sage.combinat.symmetric_group_representations import SymmetricGroupRepresentation_generic_class


def induced_representation_matrix(g, sub_rep, coset_reps):
    """
    Compute the matrix for the induced representation of g.
    
    The induced representation Ind_H^G(ρ) of a representation ρ of subgroup H
    to group G acts on the space of functions from G/H to the representation space.
    
    Parameters:
    -----------
    g : Permutation
        Element of the symmetric group S_n
    sub_rep : function
        Function that takes a permutation and returns its representation matrix
        for the subgroup representation
    coset_reps : list of Permutations
        Representatives of the left cosets G/H
        
    Returns:
    --------
    Matrix representing g in the induced representation
    
    The formula is: [Ind(g)]_{i,j} = ρ(r_i * g * r_j^{-1}) if r_i * g * r_j^{-1} ∈ H,
                                      0 otherwise
    where r_i, r_j are coset representatives and ρ is the subgroup representation.
    """
    n = len(coset_reps)
    d = sub_rep(Permutation([1])).nrows()  # dimension of sub-representation
    
    # Build block matrix
    blocks = [[None for _ in range(n)] for _ in range(n)]
    zero_block = matrix(ZZ, d, d, 0)
    
    for i, r_i in enumerate(coset_reps):
        for j, r_j in enumerate(coset_reps):
            # Compute r_i * g * r_j^{-1}
            perm = r_i * g * r_j.inverse()
            
            # Check if this permutation is in the subgroup
            # For this implementation, we'll compute the matrix
            # If it's not in H, sub_rep should handle it or we check explicitly
            try:
                blocks[i][j] = sub_rep(perm)
            except:
                blocks[i][j] = zero_block
    
    return block_matrix(blocks)


class InducedRepresentation(SymmetricGroupRepresentation_generic_class):
    """
    Class for computing induced representations from Young subgroups.
    
    Given a representation of S_k (acting on first k elements), 
    compute its induction to S_n.
    """
    
    def __init__(self, n, k, sub_representation):
        """
        Initialize induced representation from S_k to S_n.
        
        Parameters:
        -----------
        n : int
            Size of the full symmetric group
        k : int
            Size of the subgroup (k <= n)
        sub_representation : function or SymmetricGroupRepresentation
            Representation of S_k
        """
        self.n = n
        self.k = k
        self.sub_rep = sub_representation
        
        # Get coset representatives for S_n / S_k
        # S_k acts on {1, ..., k}, so cosets correspond to ways to choose
        # which k elements are acted upon
        self.coset_reps = self._compute_coset_representatives()

    def _repr_(self):
        r"""
        Return a string representation of ``self``.

        EXAMPLES::

            sage: sym_repn_3 = SymmetricGroupRepresentation([2,1])
            sage: InducedRepresentation(4,3,sym_repn_3)
            Induced representation of symmetric group of representation associated to [2,1] from S_3 to S_4
        """
        return "Induced representation of symmetric group of representation associated to partition {} from S_{} to S_{}".format(self.sub_rep._partition, self.k, self.n)

    def _compute_coset_representatives(self):
        """
        Compute representatives of left cosets S_n / S_k.
        
        We use permutations that map {1,...,k} to different k-subsets.
        """
        from sage.combinat.combination import Combinations
        
        reps = []
        for subset in Combinations(range(1, self.n + 1), self.k):
            # Create a permutation that maps {1,...,k} to subset
            perm_list = list(range(1, self.n + 1))
            
            # Put elements of subset in first k positions
            for i, elem in enumerate(subset):
                # Find where elem is and swap it to position i
                idx = perm_list.index(elem)
                perm_list[i], perm_list[idx] = perm_list[idx], perm_list[i]
            
            reps.append(Permutation(perm_list))
        
        return reps
    
    def __call__(self, g):
        """
        Compute the induced representation matrix for element g.
        
        Parameters:
        -----------
        g : Permutation or list
            Element of S_n
            
        Returns:
        --------
        Matrix in the induced representation
        """
        if not isinstance(g, Permutation):
            g = Permutation(g)
            
        n_cosets = len(self.coset_reps)
        
        if callable(self.sub_rep):
            d = self.sub_rep(Permutation(range(1, self.k + 1))).nrows()
        else:
            d = self.sub_rep(Permutation(range(1, self.k + 1))).nrows()
        
        # Build the induced representation matrix as a block matrix
        blocks = [[matrix(ZZ, d, d, 0) for _ in range(n_cosets)] 
                  for _ in range(n_cosets)]
        
        for i, r_i in enumerate(self.coset_reps):
            for j, r_j in enumerate(self.coset_reps):
                # Compute h = r_i * g * r_j^{-1}
                h = r_i * g * r_j.inverse()
                
                # Check if h stabilizes {1,...,k} (i.e., h is in S_k × S_{n-k})
                h_list = list(h)
                first_k = sorted(h_list[:self.k])
                
                if first_k == list(range(1, self.k + 1)):
                    # h acts on first k elements like an element of S_k
                    # Extract the S_k part
                    h_k = Permutation(h_list[:self.k])
                    blocks[i][j] = self._get_sub_rep_matrix(h_k)
        
        return block_matrix(blocks)
    
    def _get_sub_rep_matrix(self, perm):
        """Helper to get representation matrix from sub_rep."""
        if callable(self.sub_rep):
            return self.sub_rep(perm)
        else:
            return self.sub_rep(perm)
    
    def dimension(self):
        """Return the dimension of the induced representation."""
        from sage.combinat.combination import Combinations
        n_cosets = len(list(Combinations(self.n, self.k)))
        
        if callable(self.sub_rep):
            d_sub = self.sub_rep(Permutation(range(1, self.k + 1))).nrows()
        else:
            d_sub = self.sub_rep(Permutation(range(1, self.k + 1))).nrows()
        
        return n_cosets * d_sub


# Example usage and testing
def induced_example_usage():
    """
    Example: Induce the trivial representation of S_2 to S_3.
    """
    # Trivial representation of S_2
    def trivial_rep_s2(perm):
        return matrix(ZZ, 1, 1, [1])
    
    # Create induced representation
    ind_rep = InducedRepresentation(3, 2, trivial_rep_s2)
    
    print("Dimension of induced representation:", ind_rep.dimension())
    print("\nCoset representatives:")
    for i, r in enumerate(ind_rep.coset_reps):
        print(f"  r_{i} = {r}")
    
    print("\nRepresentation matrices:")
    for perm in Permutations(3):
        print(f"\nπ({perm}) =")
        print(ind_rep(perm))
    
    return ind_rep


if __name__ == "__main__":
    # This would run in Sage
    print("To use: create an InducedRepresentation object with your subgroup rep")
    print("Example: ind_rep = InducedRepresentation(4, 3, my_s3_representation)")
    print("Then: ind_rep(Permutation([2,1,3,4])) gives the matrix")

To use: create an InducedRepresentation object with your subgroup rep
Example: ind_rep = InducedRepresentation(4, 3, my_s3_representation)
Then: ind_rep(Permutation([2,1,3,4])) gives the matrix


In [4]:
induced_example_usage()

Dimension of induced representation: 3

Coset representatives:
  r_0 = [1, 2, 3]
  r_1 = [1, 3, 2]
  r_2 = [2, 3, 1]

Representation matrices:

π([1, 2, 3]) =
[1|0|0]
[-+-+-]
[0|1|0]
[-+-+-]
[0|0|1]

π([1, 3, 2]) =
[0|1|0]
[-+-+-]
[1|0|0]
[-+-+-]
[0|0|1]

π([2, 1, 3]) =
[1|0|0]
[-+-+-]
[0|0|1]
[-+-+-]
[0|1|0]

π([2, 3, 1]) =
[0|0|1]
[-+-+-]
[1|0|0]
[-+-+-]
[0|1|0]

π([3, 1, 2]) =
[0|1|0]
[-+-+-]
[0|0|1]
[-+-+-]
[1|0|0]

π([3, 2, 1]) =
[0|0|1]
[-+-+-]
[0|1|0]
[-+-+-]
[1|0|0]


<repr(<__main__.InducedRepresentation at 0x1213b1a50>) failed: AttributeError: 'function' object has no attribute '_partition'>

In [5]:
sym_repn_3 = SymmetricGroupRepresentation([2,1])
print(sym_repn_3(Permutation([1,2,3])))

[1 0]
[0 1]


In [6]:
sym_repn_3 = SymmetricGroupRepresentation([2,1])
ind_rep_4_3_sym21 = InducedRepresentation(4,3,sym_repn_3)
print(ind_rep_4_3_sym21)

Induced representation of symmetric group of representation associated to partition [2, 1] from S_3 to S_4


In [7]:
    # Trivial representation of S_2
    def trivial_rep_s2(perm):
        return matrix(ZZ, 1, 1, [1])
    
    # Create induced representation
    ind_rep = InducedRepresentation(3, 2, trivial_rep_s2)

In [8]:
ind_rep.dimension()

3

In [9]:
ind_rep(Permutation([1,2,3]))

[1|0|0]
[-+-+-]
[0|1|0]
[-+-+-]
[0|0|1]

In [10]:
mat_321 = ind_rep_4_3_sym21(Permutation([3,2,1])); mat_321

[-1  1| 0  0| 0  0| 0  0]
[ 0  1| 0  0| 0  0| 0  0]
[-----+-----+-----+-----]
[ 0  0| 0  0| 0  0| 0 -1]
[ 0  0| 0  0| 0  0|-1  0]
[-----+-----+-----+-----]
[ 0  0| 0  0| 0 -1| 0  0]
[ 0  0| 0  0|-1  0| 0  0]
[-----+-----+-----+-----]
[ 0  0| 0 -1| 0  0| 0  0]
[ 0  0|-1  0| 0  0| 0  0]

In [11]:
mat_321*mat_321

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]

In [14]:
from sage.combinat.permutation import Permutation
from sage.matrix.constructor import matrix
from sage.rings.integer_ring import ZZ


class RestrictedRepresentation(SymmetricGroupRepresentation_generic_class):
    r"""
    Class for computing restricted representations to Young subgroups.

    Given a representation of S_n, compute its restriction to S_k (acting on first k elements).
    """

    def __init__(self, n, k, representation):
        """
        Initialize restricted representation from S_n to S_k.

        INPUT::

        - ``n`` -- int, Size of the full symmetric group

        - ``k`` -- int, Size of the subgroup (k <= n)

        - ``representation`` -- function or SymmetricGroupRepresentation of S_n

        EXAMPLES::

            sage: sym_repn_4 = SymmetricGroupRepresentation([2,2])
            sage: res_rep_4_3 = RestrictedRepresentation(4, 3, sym_repn_4)
            sage: res_rep_4_3.dimension()
            2
            sage: sym_repn_3 = SymmetricGroupRepresentation([3])
            sage: res_rep_3_2 = RestrictedRepresentation(3, 2, sym_repn_3)
            sage: res_rep_3_2.dimension()
            1
        """
        if k > n:
            raise ValueError(f"Subgroup size k={k} cannot be larger than group size n={n}")

        self.n = n
        self.k = k
        self.representation = representation

    def _repr_(self):
        r"""
        Return a string representation of ``self``.

        EXAMPLES::

            sage: sym_repn_4 = SymmetricGroupRepresentation([2,2])
            sage: RestrictedRepresentation(4, 3, sym_repn_4)
            Restricted representation of symmetric group of representation associated to partition [2, 2] from S_4 to S_3
        """
        if hasattr(self.representation, '_partition'):
            return "Restricted representation of symmetric group of representation associated to partition {} from S_{} to S_{}".format(
                self.representation._partition, self.n, self.k)
        else:
            return "Restricted representation from S_{} to S_{}".format(self.n, self.k)

    def __call__(self, g):
        r"""
        Returns the restricted representation matrix for element g in S_k.

        The restriction simply evaluates the larger representation on an element
        of S_k, viewed as an element of S_n that fixes {k+1, ..., n}.

        INPUT:

        - ``g`` -- Permutation or list, Element of S_k

        EXAMPLES::

            sage: sym_repn_4 = SymmetricGroupRepresentation([2,2])
            sage: res_rep_4_3 = RestrictedRepresentation(4, 3, sym_repn_4)
            sage: res_rep_4_3([2,1,3])
            [ 1  0]
            [ 0 -1]
        """
        if not isinstance(g, Permutation):
            g = Permutation(g)

        # Check that g is indeed in S_k
        if len(g) > self.k:
            raise ValueError(f"Permutation {g} has length {len(g)} > k={self.k}")

        # Embed g into S_n by extending it to fix elements {k+1, ..., n}
        g_list = list(g)
        
        # Pad with identity on remaining elements
        g_extended = g_list + list(range(len(g_list) + 1, self.n + 1))
        g_in_Sn = Permutation(g_extended)

        # Evaluate the representation on the extended permutation
        if callable(self.representation):
            return self.representation(g_in_Sn)
        else:
            return self.representation(g_in_Sn)

    def dimension(self):
        r"""
        Return the dimension of the restricted representation.

        The dimension is the same as the dimension of the original representation.

        EXAMPLES::

            sage: sym_repn_4 = SymmetricGroupRepresentation([2,2])
            sage: res_rep_4_3 = RestrictedRepresentation(4, 3, sym_repn_4)
            sage: res_rep_4_3.dimension()
            2
        """
        # Get dimension by evaluating on identity
        identity_in_Sk = Permutation(range(1, self.k + 1))
        return self(identity_in_Sk).nrows()


# Example usage and testing
def restricted_example_usage():
    """
    Example: Restrict a representation of S_3 to S_2.
    """
    # Create a representation of S_3
    sym_rep_s3 = SymmetricGroupRepresentation([2, 1])
    
    # Restrict it to S_2
    res_rep = RestrictedRepresentation(3, 2, sym_rep_s3)
    
    print("Dimension of restricted representation:", res_rep.dimension())
    print("\nRepresentation matrices for S_2 elements:")
    
    from sage.combinat.permutation import Permutations
    for perm in Permutations(2):
        print(f"\nρ({perm}) =")
        print(res_rep(perm))
    
    return res_rep


if __name__ == "__main__":
    print("To use: create a RestrictedRepresentation object")
    print("Example: res_rep = RestrictedRepresentation(4, 3, my_s4_representation)")
    print("Then: res_rep(Permutation([2,1,3])) gives the matrix")

To use: create a RestrictedRepresentation object
Example: res_rep = RestrictedRepresentation(4, 3, my_s4_representation)
Then: res_rep(Permutation([2,1,3])) gives the matrix


In [15]:
restricted_example_usage()

Dimension of restricted representation: 2

Representation matrices for S_2 elements:

ρ([1, 2]) =
[1 0]
[0 1]

ρ([2, 1]) =
[ 0 -1]
[-1  0]


Restricted representation of symmetric group of representation associated to partition [2, 1] from S_3 to S_2

In [16]:
from sage.combinat.permutation import Permutation, Permutations
from sage.groups.perm_gps.permgroup_named import SymmetricGroup
from sage.rings.rational_field import QQ


def character_inner_product(chi1, chi2, group):
    r"""
    Compute the inner product of two characters over a group.
    
    For finite groups, this is:
    ⟨χ₁, χ₂⟩ = (1/|G|) * Σ_{g ∈ G} χ₁(g) * conj(χ₂(g))
    
    For real-valued characters (which symmetric group characters are),
    this simplifies to:
    ⟨χ₁, χ₂⟩ = (1/|G|) * Σ_{g ∈ G} χ₁(g) * χ₂(g)
    
    INPUT:
    
    - ``chi1`` -- function that takes a permutation and returns a scalar (character value)
    - ``chi2`` -- function that takes a permutation and returns a scalar (character value)
    - ``group`` -- iterable of group elements
    
    OUTPUT:
    
    The inner product as a rational number
    """
    group_list = list(group)
    n = len(group_list)
    
    total = sum(chi1(g) * chi2(g) for g in group_list)
    
    return QQ(total) / QQ(n)


def verify_frobenius_reciprocity(n, k, rep_H, rep_G, verbose=True):
    r"""
    Verify Frobenius reciprocity for symmetric group representations.
    
    Tests the formula:
    ⟨Ind_H^G(χ_H), χ_G⟩_G = ⟨χ_H, Res_G^H(χ_G)⟩_H
    
    where χ_H is the character of rep_H and χ_G is the character of rep_G.
    
    INPUT:
    
    - ``n`` -- size of full symmetric group S_n
    - ``k`` -- size of subgroup S_k
    - ``rep_H`` -- representation of S_k
    - ``rep_G`` -- representation of S_n
    - ``verbose`` -- whether to print detailed information
    
    OUTPUT:
    
    True if Frobenius reciprocity holds (up to numerical precision), False otherwise
    """
    from sage.combinat.symmetric_group_representations import SymmetricGroupRepresentation
    
    # Create induced and restricted representations
    # Assuming InducedRepresentation and RestrictedRepresentation are available
    ind_rep = InducedRepresentation(n, k, rep_H)
    res_rep = RestrictedRepresentation(n, k, rep_G)
    
    # Define characters as trace functions
    def chi_H(g):
        return rep_H(g).trace()
    
    def chi_G(g):
        return rep_G(g).trace()
    
    def chi_ind(g):
        return ind_rep(g).trace()
    
    def chi_res(g):
        return res_rep(g).trace()
    
    # Compute inner products
    # Left side: ⟨Ind(χ_H), χ_G⟩_G
    G = Permutations(n)
    left_side = character_inner_product(chi_ind, chi_G, G)
    
    # Right side: ⟨χ_H, Res(χ_G)⟩_H
    H = Permutations(k)
    right_side = character_inner_product(chi_H, chi_res, H)
    
    if verbose:
        print(f"Verifying Frobenius reciprocity for S_{k} ⊂ S_{n}")
        print(f"Representation of S_{k}: {rep_H}")
        print(f"Representation of S_{n}: {rep_G}")
        print(f"\nLeft side  ⟨Ind(χ), ψ⟩_G = {left_side}")
        print(f"Right side ⟨χ, Res(ψ)⟩_H = {right_side}")
        print(f"\nDifference: {abs(left_side - right_side)}")
    
    # Check equality (allowing for small numerical errors)
    return abs(left_side - right_side) < 1e-10


def verify_dimension_formulas(n, k, rep_H, verbose=True):
    r"""
    Verify dimension formulas for induced representations.
    
    The dimension of Ind_H^G(V) should be [G:H] * dim(V)
    where [G:H] is the index of H in G.
    
    For S_k ⊂ S_n, the index is n!/k! = n*(n-1)*...*(k+1) = P(n,k)
    which is also C(n,k) * k! where we choose k elements then arrange them.
    But actually for our embedding, the index is C(n,k) = n!/(k!(n-k)!)
    
    INPUT:
    
    - ``n`` -- size of full symmetric group
    - ``k`` -- size of subgroup
    - ``rep_H`` -- representation of S_k
    - ``verbose`` -- whether to print information
    """
    from sage.combinat.combination import Combinations
    
    ind_rep = InducedRepresentation(n, k, rep_H)
    
    # Compute expected dimension
    index = len(list(Combinations(n, k)))  # This is C(n,k)
    dim_H = rep_H(Permutation(range(1, k+1))).nrows()
    expected_dim = index * dim_H
    
    actual_dim = ind_rep.dimension()
    
    if verbose:
        print(f"\nDimension formula verification:")
        print(f"Index [G:H] = C({n},{k}) = {index}")
        print(f"dim(V_H) = {dim_H}")
        print(f"Expected dim(Ind(V_H)) = {expected_dim}")
        print(f"Actual dim(Ind(V_H)) = {actual_dim}")
        print(f"Match: {expected_dim == actual_dim}")
    
    return expected_dim == actual_dim


def run_examples():
    r"""
    Run examples verifying Frobenius reciprocity and related formulas.
    """
    from sage.combinat.symmetric_group_representations import SymmetricGroupRepresentation
    
    print("="*70)
    print("FROBENIUS RECIPROCITY VERIFICATION")
    print("="*70)
    
    # Example 1: Trivial representation of S_2, standard rep of S_3
    print("\n" + "="*70)
    print("Example 1: S_2 ⊂ S_3")
    print("="*70)
    
    rep_S2 = SymmetricGroupRepresentation([2])  # Trivial rep of S_2
    rep_S3 = SymmetricGroupRepresentation([2,1])  # Standard rep of S_3
    
    result1 = verify_frobenius_reciprocity(3, 2, rep_S2, rep_S3)
    print(f"\nFrobenius reciprocity holds: {result1}")
    
    verify_dimension_formulas(3, 2, rep_S2)
    
    # Example 2: Standard rep of S_3, rep of S_4
    print("\n" + "="*70)
    print("Example 2: S_3 ⊂ S_4")
    print("="*70)
    
    rep_S3_std = SymmetricGroupRepresentation([2,1])  # Standard rep of S_3
    rep_S4 = SymmetricGroupRepresentation([3,1])  # A rep of S_4
    
    result2 = verify_frobenius_reciprocity(4, 3, rep_S3_std, rep_S4)
    print(f"\nFrobenius reciprocity holds: {result2}")
    
    verify_dimension_formulas(4, 3, rep_S3_std)
    
    # Example 3: Sign representation
    print("\n" + "="*70)
    print("Example 3: Sign representation S_2 ⊂ S_3")
    print("="*70)
    
    rep_S2_sign = SymmetricGroupRepresentation([1,1])  # Sign rep of S_2
    rep_S3_triv = SymmetricGroupRepresentation([3])  # Trivial rep of S_3
    
    result3 = verify_frobenius_reciprocity(3, 2, rep_S2_sign, rep_S3_triv)
    print(f"\nFrobenius reciprocity holds: {result3}")
    
    print("\n" + "="*70)
    print("All tests completed!")
    print("="*70)


if __name__ == "__main__":
    print("Frobenius Reciprocity Verification Tools")
    print("\nTo run examples: run_examples()")
    print("\nTo verify specific case:")
    print("  verify_frobenius_reciprocity(n, k, rep_H, rep_G)")

Frobenius Reciprocity Verification Tools

To run examples: run_examples()

To verify specific case:
  verify_frobenius_reciprocity(n, k, rep_H, rep_G)


In [17]:
run_examples()

FROBENIUS RECIPROCITY VERIFICATION

Example 1: S_2 ⊂ S_3
Verifying Frobenius reciprocity for S_2 ⊂ S_3
Representation of S_2: Specht representation of the symmetric group corresponding to [2]
Representation of S_3: Specht representation of the symmetric group corresponding to [2, 1]

Left side  ⟨Ind(χ), ψ⟩_G = 1
Right side ⟨χ, Res(ψ)⟩_H = 1

Difference: 0

Frobenius reciprocity holds: True

Dimension formula verification:
Index [G:H] = C(3,2) = 3
dim(V_H) = 1
Expected dim(Ind(V_H)) = 3
Actual dim(Ind(V_H)) = 3
Match: True

Example 2: S_3 ⊂ S_4
Verifying Frobenius reciprocity for S_3 ⊂ S_4
Representation of S_3: Specht representation of the symmetric group corresponding to [2, 1]
Representation of S_4: Specht representation of the symmetric group corresponding to [3, 1]

Left side  ⟨Ind(χ), ψ⟩_G = 1
Right side ⟨χ, Res(ψ)⟩_H = 1

Difference: 0

Frobenius reciprocity holds: True

Dimension formula verification:
Index [G:H] = C(4,3) = 4
dim(V_H) = 2
Expected dim(Ind(V_H)) = 8
Actual dim(In